# Document Q&A System with AWS Bedrock

This notebook demonstrates how to build a question-answering system over documents using:
- **AWS Bedrock** (Amazon Nova Lite) for LLM capabilities
- **Wikipedia API** to fetch current events data
- **Embeddings and Vector Search** for document retrieval

Based on concepts from LangChain's document Q&A approach.

## Setup and Installation

First, let's install the required packages.

In [ ]:
!pip install boto3 wikipedia-api numpy scikit-learn requests beautifulsoup4 -q

## Import Dependencies

In [ ]:
import boto3
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import requests
from bs4 import BeautifulSoup
from typing import List, Dict, Tuple
import re
from datetime import datetime

## Configure AWS Bedrock

Set up your AWS credentials and initialize the Bedrock client.

In [ ]:
# Import Google Colab userdata for secure credential access
from google.colab import userdata

# Configure your AWS credentials using Colab secrets
AWS_ACCESS_KEY_ID = userdata.get('awsid')  # Set this in Colab secrets
AWS_SECRET_ACCESS_KEY = userdata.get('awssecret')  # Set this in Colab secrets
AWS_REGION = "us-east-1"  # Change if needed

# Initialize Bedrock client
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name=AWS_REGION,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)

print("✓ AWS Bedrock client initialized")

## Fetch Current Events from Wikipedia

We'll scrape Wikipedia's Current Events portal to get recent news.

In [ ]:
def fetch_wikipedia_current_events(max_days=3):
    """
    Fetch current events from Wikipedia's Portal:Current_events
    
    Args:
        max_days: Number of recent days to fetch (default: 3)
    
    Returns:
        List of dictionaries with date and events
    """
    url = "https://en.wikipedia.org/wiki/Portal:Current_events"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        events_by_date = []
        
        # Find all date headers (h3 tags with date class)
        date_sections = soup.find_all('h3')
        
        for i, date_section in enumerate(date_sections[:max_days]):
            date_text = date_section.get_text().strip()
            
            # Find the content div following this header
            content_div = date_section.find_next_sibling('div')
            
            if content_div:
                # Extract all list items (events)
                events = []
                
                # Find all categories and their events
                categories = content_div.find_all('dl')
                
                for category in categories:
                    # Get category name
                    category_name = category.find('dt')
                    if category_name:
                        category_text = category_name.get_text().strip()
                        
                        # Get all events in this category
                        event_items = category.find_all('dd')
                        
                        for event in event_items:
                            event_text = event.get_text().strip()
                            # Clean up the text
                            event_text = re.sub(r'\s+', ' ', event_text)
                            
                            if event_text:
                                events.append({
                                    'category': category_text,
                                    'event': event_text
                                })
                
                if events:
                    events_by_date.append({
                        'date': date_text,
                        'events': events
                    })
        
        return events_by_date
    
    except Exception as e:
        print(f"Error fetching Wikipedia events: {e}")
        return []

# Fetch current events
print("Fetching current events from Wikipedia...")
events_data = fetch_wikipedia_current_events(max_days=3)

print(f"\n✓ Fetched events from {len(events_data)} days")
for day_data in events_data:
    print(f"  - {day_data['date']}: {len(day_data['events'])} events")

## Preview the Fetched Data

In [ ]:
# Display sample events
if events_data:
    print("\n=== Sample Events ===")
    first_day = events_data[0]
    print(f"\nDate: {first_day['date']}")
    print("\nFirst 5 events:")
    
    for i, event in enumerate(first_day['events'][:5], 1):
        print(f"\n{i}. Category: {event['category']}")
        print(f"   Event: {event['event'][:200]}..." if len(event['event']) > 200 else f"   Event: {event['event']}")
else:
    print("No events fetched. Please check your internet connection.")

## Create Document Chunks

Convert the events into document chunks for retrieval.

In [ ]:
def create_document_chunks(events_data):
    """
    Create document chunks from events data.
    Each chunk contains context about date, category, and event.
    """
    chunks = []
    
    for day_data in events_data:
        date = day_data['date']
        
        for event in day_data['events']:
            # Create a comprehensive chunk with context
            chunk_text = f"""Date: {date}
Category: {event['category']}
Event: {event['event']}"""
            
            chunks.append({
                'text': chunk_text,
                'date': date,
                'category': event['category'],
                'event': event['event']
            })
    
    return chunks

# Create document chunks
document_chunks = create_document_chunks(events_data)

print(f"\n✓ Created {len(document_chunks)} document chunks")
print(f"\nSample chunk:")
print("="*50)
print(document_chunks[0]['text'])
print("="*50)

## AWS Bedrock Helper Functions

Functions to interact with Amazon Nova Lite via Bedrock.

In [ ]:
def get_embedding_bedrock(text: str) -> List[float]:
    """
    Get embeddings using Amazon Titan Embeddings via Bedrock.
    """
    try:
        body = json.dumps({
            "inputText": text
        })
        
        response = bedrock_runtime.invoke_model(
            modelId='amazon.titan-embed-text-v1',
            body=body,
            contentType='application/json',
            accept='application/json'
        )
        
        response_body = json.loads(response['body'].read())
        return response_body['embedding']
    
    except Exception as e:
        print(f"Error getting embedding: {e}")
        # Return a zero vector as fallback
        return [0.0] * 1536


def invoke_nova(prompt: str, system_prompt: str = "", max_tokens: int = 2000) -> str:
    """
    Invoke Amazon Nova Lite via AWS Bedrock.
    """
    try:
        # Combine system and user prompts for Nova Lite
        if system_prompt:
            full_prompt = f"{system_prompt}\n\nHuman: {prompt}\n\nAssistant:"
        else:
            full_prompt = f"Human: {prompt}\n\nAssistant:"
        
        body = {
            "inputText": full_prompt,
            "textGenerationConfig": {
                "maxTokenCount": max_tokens,
                "temperature": 0.7,
                "topP": 0.9
            }
        }
        
        response = bedrock_runtime.invoke_model(
            modelId='amazon.nova-lite-v1:0',
            body=json.dumps(body),
            contentType='application/json',
            accept='application/json'
        )
        
        response_body = json.loads(response['body'].read())
        return response_body['results'][0]['outputText']
    
    except Exception as e:
        print(f"Error invoking Nova Lite: {e}")
        return f"Error: {str(e)}"

print("✓ Helper functions defined")

## Create Vector Store

Create embeddings for all document chunks and build an in-memory vector store.

In [ ]:
class SimpleVectorStore:
    """
    A simple in-memory vector store using cosine similarity.
    Similar to LangChain's DocArrayInMemorySearch.
    """
    
    def __init__(self):
        self.documents = []
        self.embeddings = []
    
    def add_documents(self, chunks: List[Dict]):
        """
        Add documents and create embeddings.
        """
        print(f"Creating embeddings for {len(chunks)} documents...")
        
        for i, chunk in enumerate(chunks):
            if i % 10 == 0:
                print(f"  Processing chunk {i+1}/{len(chunks)}...")
            
            embedding = get_embedding_bedrock(chunk['text'])
            self.documents.append(chunk)
            self.embeddings.append(embedding)
        
        self.embeddings = np.array(self.embeddings)
        print(f"✓ Created embeddings with shape: {self.embeddings.shape}")
    
    def similarity_search(self, query: str, k: int = 4) -> List[Dict]:
        """
        Find the k most similar documents to the query.
        """
        # Get query embedding
        query_embedding = np.array(get_embedding_bedrock(query)).reshape(1, -1)
        
        # Calculate cosine similarities
        similarities = cosine_similarity(query_embedding, self.embeddings)[0]
        
        # Get top k indices
        top_k_indices = np.argsort(similarities)[-k:][::-1]
        
        # Return top k documents with scores
        results = []
        for idx in top_k_indices:
            doc = self.documents[idx].copy()
            doc['similarity_score'] = float(similarities[idx])
            results.append(doc)
        
        return results

# Create and populate the vector store
print("\nInitializing vector store...")
vector_store = SimpleVectorStore()
vector_store.add_documents(document_chunks)
print("\n✓ Vector store ready!")

## Test Document Retrieval

Let's test the similarity search functionality.

In [ ]:
# Test retrieval with a sample query
test_query = "What happened in sports recently?"

print(f"Query: {test_query}")
print("\nRetrieved documents:")
print("="*70)

retrieved_docs = vector_store.similarity_search(test_query, k=3)

for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n{i}. Similarity Score: {doc['similarity_score']:.4f}")
    print(f"   Date: {doc['date']}")
    print(f"   Category: {doc['category']}")
    print(f"   Event: {doc['event'][:150]}..." if len(doc['event']) > 150 else f"   Event: {doc['event']}")
    print("-"*70)

## Build the Q&A Chain

Create the complete question-answering system.

In [ ]:
class DocumentQAChain:
    """
    A question-answering chain over documents.
    Similar to LangChain's RetrievalQA chain.
    """
    
    def __init__(self, vector_store: SimpleVectorStore, verbose: bool = False):
        self.vector_store = vector_store
        self.verbose = verbose
    
    def run(self, query: str, k: int = 4) -> Dict:
        """
        Run the QA chain on a query.
        """
        if self.verbose:
            print(f"\n{'='*70}")
            print(f"QUERY: {query}")
            print(f"{'='*70}")
        
        # Step 1: Retrieve relevant documents
        if self.verbose:
            print(f"\n[RETRIEVAL] Searching for {k} most relevant documents...")
        
        retrieved_docs = self.vector_store.similarity_search(query, k=k)
        
        if self.verbose:
            print(f"\n[RETRIEVED DOCUMENTS]")
            for i, doc in enumerate(retrieved_docs, 1):
                print(f"\n  Document {i} (Score: {doc['similarity_score']:.4f}):")
                print(f"  {doc['text'][:200]}..." if len(doc['text']) > 200 else f"  {doc['text']}")
        
        # Step 2: Combine documents into context
        context = "\n\n".join([doc['text'] for doc in retrieved_docs])
        
        # Step 3: Create prompt for Nova Lite
        system_prompt = """You are a helpful assistant that answers questions based on the provided context. 
Use only the information from the context to answer the question. 
If you cannot answer based on the context, say so."""
        
        user_prompt = f"""Context:
{context}

Question: {query}

Answer:"""
        
        if self.verbose:
            print(f"\n[PROMPT TO LLM]")
            print(f"System: {system_prompt}")
            print(f"\nUser: {user_prompt[:500]}..." if len(user_prompt) > 500 else f"\nUser: {user_prompt}")
        
        # Step 4: Get answer from Nova Lite
        if self.verbose:
            print(f"\n[INVOKING NOVA LITE]...")
        
        answer = invoke_nova(user_prompt, system_prompt=system_prompt)
        
        if self.verbose:
            print(f"\n[RESPONSE]")
            print(f"{answer}")
            print(f"\n{'='*70}")
        
        return {
            'query': query,
            'answer': answer,
            'source_documents': retrieved_docs
        }

# Create the QA chain
qa_chain = DocumentQAChain(vector_store, verbose=True)

print("\n✓ QA Chain created!")

## Run Question Answering

Now let's ask questions about the current events!

In [ ]:
# Example 1: Sports question
result = qa_chain.run("What sports events happened recently?")

In [ ]:
# Display the answer nicely
from IPython.display import display, Markdown

display(Markdown(f"**Question:** {result['query']}"))
display(Markdown(f"**Answer:** {result['answer']}"))

In [ ]:
# Example 2: Politics question
result = qa_chain.run("Tell me about recent political events")

In [ ]:
display(Markdown(f"**Question:** {result['query']}"))
display(Markdown(f"**Answer:** {result['answer']}"))

In [ ]:
# Example 3: Disaster/accident question
result = qa_chain.run("Were there any disasters or accidents reported?")

In [ ]:
display(Markdown(f"**Question:** {result['query']}"))
display(Markdown(f"**Answer:** {result['answer']}"))

## Interactive Q&A

Try your own questions!

In [ ]:
# Turn off verbose mode for cleaner output
qa_chain.verbose = False

def ask_question(question: str):
    """
    Helper function to ask questions with nice formatting.
    """
    print("\n" + "="*70)
    print(f"Q: {question}")
    print("="*70)
    
    result = qa_chain.run(question)
    
    print(f"\nA: {result['answer']}")
    print("\nSources:")
    for i, doc in enumerate(result['source_documents'][:2], 1):
        print(f"  {i}. {doc['date']} - {doc['category']}")
    print("="*70)
    
    return result

# Try some questions
ask_question("What happened in the Gaza war?")

In [ ]:
ask_question("Tell me about business and economy news")

In [ ]:
ask_question("What international relations events occurred?")

In [ ]:
# Your custom question
your_question = "What elections or political changes happened recently?"
ask_question(your_question)

## Evaluation: Generate Test Examples

Let's use Nova Lite to automatically generate question-answer pairs for evaluation.

In [ ]:
def generate_qa_examples(documents: List[Dict], num_examples: int = 5) -> List[Dict]:
    """
    Generate question-answer pairs from documents using Nova Lite.
    """
    examples = []
    
    # Sample documents
    sampled_docs = np.random.choice(documents, min(num_examples, len(documents)), replace=False)
    
    print(f"Generating {num_examples} Q&A examples...\n")
    
    for i, doc in enumerate(sampled_docs, 1):
        print(f"Generating example {i}/{len(sampled_docs)}...")
        
        prompt = f"""Based on the following event, create a question and answer pair.

Event:
{doc['text']}

Generate:
1. A specific question that can be answered from this event
2. A concise answer to that question

Format your response as:
QUESTION: [your question]
ANSWER: [your answer]"""
        
        response = invoke_nova(prompt)
        
        # Parse the response
        try:
            question_match = re.search(r'QUESTION:\s*(.+?)(?=ANSWER:|$)', response, re.DOTALL)
            answer_match = re.search(r'ANSWER:\s*(.+)', response, re.DOTALL)
            
            if question_match and answer_match:
                question = question_match.group(1).strip()
                answer = answer_match.group(1).strip()
                
                examples.append({
                    'question': question,
                    'ground_truth_answer': answer,
                    'source_document': doc
                })
        except Exception as e:
            print(f"  Error parsing response: {e}")
            continue
    
    print(f"\n✓ Generated {len(examples)} examples")
    return examples

# Generate test examples
test_examples = generate_qa_examples(document_chunks, num_examples=5)

In [ ]:
# Display generated examples
print("\n=== Generated Test Examples ===")
for i, example in enumerate(test_examples, 1):
    print(f"\nExample {i}:")
    print(f"Q: {example['question']}")
    print(f"A: {example['ground_truth_answer']}")
    print("-"*70)

## Run Evaluation

Test the QA system on generated examples and evaluate using Nova Lite.

In [ ]:
def evaluate_qa_system(qa_chain: DocumentQAChain, test_examples: List[Dict]) -> List[Dict]:
    """
    Evaluate the QA system on test examples.
    """
    results = []
    
    print(f"Evaluating QA system on {len(test_examples)} examples...\n")
    
    for i, example in enumerate(test_examples, 1):
        print(f"Testing example {i}/{len(test_examples)}...")
        
        # Get prediction from QA chain
        prediction = qa_chain.run(example['question'])
        
        # Evaluate with Nova Lite
        eval_prompt = f"""Compare the following predicted answer with the ground truth answer.

Question: {example['question']}

Ground Truth Answer: {example['ground_truth_answer']}

Predicted Answer: {prediction['answer']}

Does the predicted answer correctly answer the question based on the ground truth?
Respond with only: CORRECT or INCORRECT
Then provide a brief explanation."""
        
        evaluation = invoke_nova(eval_prompt)
        
        grade = "CORRECT" if "CORRECT" in evaluation.split()[0].upper() else "INCORRECT"
        
        results.append({
            'question': example['question'],
            'ground_truth': example['ground_truth_answer'],
            'prediction': prediction['answer'],
            'grade': grade,
            'evaluation': evaluation
        })
    
    print(f"\n✓ Evaluation complete")
    return results

# Run evaluation
evaluation_results = evaluate_qa_system(qa_chain, test_examples)

In [ ]:
# Display evaluation results
print("\n" + "="*70)
print("EVALUATION RESULTS")
print("="*70)

correct_count = sum(1 for r in evaluation_results if r['grade'] == 'CORRECT')
total_count = len(evaluation_results)

print(f"\nAccuracy: {correct_count}/{total_count} ({100*correct_count/total_count:.1f}%)\n")

for i, result in enumerate(evaluation_results, 1):
    print(f"\nExample {i}:")
    print(f"Question: {result['question']}")
    print(f"\nGround Truth: {result['ground_truth']}")
    print(f"\nPrediction: {result['prediction']}")
    print(f"\nGrade: {result['grade']}")
    print(f"\nEvaluation: {result['evaluation']}")
    print("-"*70)

## Summary

In this notebook, we've demonstrated:

1. **Document Loading**: Fetched current events from Wikipedia
2. **Embeddings**: Created vector representations using Amazon Titan Embeddings
3. **Vector Store**: Built an in-memory vector store for similarity search
4. **Q&A Chain**: Implemented a retrieval-based question answering system
5. **Evaluation**: Generated test cases and evaluated the system using Nova Lite

### Key Concepts:

- **Embeddings** capture semantic meaning of text
- **Vector similarity search** retrieves relevant documents
- **RAG (Retrieval Augmented Generation)** combines retrieval with generation
- **LLM-based evaluation** uses language models to assess quality

### Next Steps:

- Experiment with different retrieval parameters (k value)
- Try different prompting strategies
- Add more sophisticated chunking strategies
- Implement other chain types (map-reduce, refine, etc.)